# Week 4 Homework — RPC Semantics, Timeouts, Retries, Idempotency

Instructions:
- Complete all tasks in this notebook. Use Python 3.9+.
- Provide code, outputs, and concise analysis below each task.
- You may adapt code from the examples but explain your changes.

Grading focuses on correctness, robustness (timeouts/retries/backoff), and clarity of reasoning about semantics.

---

## Task 1 — Implement JSON-RPC `inc` and `get` with idempotency
Build a small RPC server and client using length-prefixed JSON frames.

Requirements:
- Methods: `inc(key, delta)` and `get(key)`
- Idempotency: cache responses by request `id` for ~10s window
- Client implements a per-call timeout, up to 3 retries with exponential backoff + jitter

Provide a short demo calling `inc` several times and reading back with `get`.


In [ ]:
import socket, struct, threading, time, json, random, uuid
from contextlib import closing


def send_frame(sock, b: bytes):
    sock.sendall(struct.pack('!I', len(b)) + b)


def recv_all(sock, n: int) -> bytes:
    buf = bytearray()
    while len(buf) < n:
        chunk = sock.recv(n - len(buf))
        if not chunk:
            raise ConnectionError('socket closed')
        buf.extend(chunk)
    return bytes(buf)


def recv_frame(sock) -> bytes:
    (length,) = struct.unpack('!I', recv_all(sock, 4))
    return recv_all(sock, length)


class RpcServer:
    def __init__(self, addr=('127.0.0.1', 50888)):
        self.addr = addr
        self.state = {}
        self.cache = {}
        self.cache_ttl = 10.0
        self.running = threading.Event()

    def start(self):
        self.running.set()
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind(self.addr)
        self.sock.listen(64)
        threading.Thread(target=self._accept_loop, daemon=True).start()

    def stop(self):
        self.running.clear()
        try: self.sock.close()
        except Exception: pass

    def _accept_loop(self):
        while self.running.is_set():
            try:
                c, _ = self.sock.accept()
            except OSError:
                break
            threading.Thread(target=self._client_loop, args=(c,), daemon=True).start()

    def _client_loop(self, c):
        try:
            while self.running.is_set():
                req = json.loads(recv_frame(c))
                rid = req.get('id')
                now = time.time()
                entry = self.cache.get(rid)
                if entry and entry[0] > now:
                    send_frame(c, entry[1])
                    continue
                resp = self._handle(req)
                data = json.dumps(resp).encode('utf-8')
                self.cache[rid] = (now + self.cache_ttl, data)
                send_frame(c, data)
        except Exception:
            pass
        finally:
            try: c.close()
            except Exception: pass

    def _handle(self, req: dict) -> dict:
        rid = req.get('id')
        method = req.get('method')
        p = req.get('params') or {}
        try:
            if method == 'inc':
                key = p['key']; delta = int(p.get('delta', 1))
                self.state[key] = self.state.get(key, 0) + delta
                return {"id": rid, "result": {"key": key, "value": self.state[key]}}
            if method == 'get':
                key = p['key']
                return {"id": rid, "result": {"key": key, "value": self.state.get(key, 0)}}
            return {"id": rid, "error": {"code": -32601, "message": "Method not found"}}
        except Exception as e:
            return {"id": rid, "error": {"code": -32000, "message": str(e)}}


class RpcClient:
    def __init__(self, addr=('127.0.0.1', 50888), timeout=0.25, max_retries=3, backoff_base=0.05, factor=2.0, jitter=0.1):
        self.addr = addr
        self.timeout = timeout
        self.max_retries = max_retries
        self.backoff_base = backoff_base
        self.factor = factor
        self.jitter = jitter

    def call(self, method: str, params: dict):
        rid = str(uuid.uuid4())
        payload = json.dumps({"id": rid, "method": method, "params": params}).encode('utf-8')
        attempt = 0
        while attempt <= self.max_retries:
            attempt += 1
            try:
                with closing(socket.create_connection(self.addr, timeout=self.timeout)) as s:
                    s.settimeout(self.timeout)
                    send_frame(s, payload)
                    resp = json.loads(recv_frame(s))
                    if resp.get('id') == rid:
                        return resp
            except Exception:
                delay = self.backoff_base * (self.factor ** (attempt-1)) * (1.0 + self.jitter*random.random())
                time.sleep(delay)
        raise TimeoutError(f"RPC {method} failed after {attempt} attempts")

# Demo
server = RpcServer(); server.start()
client = RpcClient()
for i in range(3):
    print(client.call('inc', {"key": "counter", "delta": 2}))
print(client.call('get', {"key": "counter"}))
server.stop()


### Analysis (Task 1)
Explain how idempotency is ensured and how the client’s retry/backoff policy interacts with server-side caching.

---

## Task 2 — Timeouts and tail latency
Inject server delay and vary client timeout/backoff to explore trade-offs.

Procedure:
- Modify your server to sleep `delay_ms` before replies
- Run with `delay_ms` in {0, 20, 50, 120}
- For each, measure success rate and p95/p99 latency under 100 calls

Provide a short table of results and discuss the impact of aggressive vs conservative timeouts.


### Analysis (Task 2)
Discuss:
- When do retries help vs harm? (Consider retry storms)
- How do you pick reasonable timeouts without global clocks?

---

## Task 3 — Idempotent vs non-idempotent handlers
Design a non-idempotent method (e.g., `append` to a list) and show how duplicates cause unintended effects.

Then redesign to be idempotent (e.g., use an idempotency key and set semantics) and demonstrate correct behavior under duplicate calls.


### Analysis (Task 3)
Explain the redesign and the correctness argument.

---

## Task 4 — Backpressure (design)
Propose a simple backpressure mechanism for the server (e.g., a bounded request queue that returns an error when full). Explain how this differs from client-side backoff and why both are needed.
